# Ручка 3. Summary

Делает краткое саммари всего конспекта (на вход идет весь файл в обсидиане, можно добавить передачу всей папки)

In [1]:
from langchain.chat_models.gigachat import GigaChat
from dotenv import load_dotenv
import os

load_dotenv()

GIGACHAT_CREDENTIALS = os.environ.get("GIGACHAT_CREDENTIALS")
SERPER_API_KEY = os.environ.get("SERPER_API_KEY")

llm = GigaChat(
    credentials=GIGACHAT_CREDENTIALS,
    model="GigaChat-Pro",
    verify_ssl_certs=True,
    scope="GIGACHAT_API_CORP",
    profanity_check=False
)

In [2]:
# Загружаем конспект
from langchain_community.document_loaders.text import TextLoader

notes_path = "./notes/small_note.md"

loader = TextLoader(notes_path, "utf-8")
docs = loader.load()

print(docs[0].page_content)

# Используем Stuff Documents Chain + Summarization Checker

In [3]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMSummarizationCheckerChain

def stuff_summary(docs, checker: bool=False, max_checks:int=2):
     prompt_template = """Выдели клюбючевые моменты в тексте.
     Оформи ответ в виде списка.
     Текст: "{context}"
     Ключевые моменты:"""
     prompt = PromptTemplate.from_template(prompt_template)
     stuff_chain = create_stuff_documents_chain(llm, prompt)
     res = stuff_chain.invoke({"context": docs})
     if not checker:
          return res
     else:
          checker_chain = LLMSummarizationCheckerChain.from_llm(llm, max_checks=max_checks)
          return checker_chain.invoke(res)
          

print(stuff_summary(docs, True, 2))

c:\Users\nonam\Documents\python_local\obsidian-gigachat\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


- Влияние параметров пара на показатели комбинированной установки с простым паровым контуром
- Максимальная мощность паровой турбины достигается при максимальных параметрах пара
- Мощность паровой турбины зависит от расхода пара, располагаемого перепада энтальпий на турбину и КПД паровой турбины
- Относительный расход пара определяется уравнением теплового баланса испарительной и перегревательной поверхностей котла-утилизатора
- Температура пара перед паровой турбиной принимается максимально возможной для уменьшения потерь от влажности и повышения КПД турбины
- С увеличением давления пара относительный расход пара снижается из-за снижения удельной теплоты парообразования
- С ростом давления растет температура уходящих газов и располагаемый перепад энтальпий на паровую турбину
- Совместное влияние параметров на величину мощности паровой турбины приводит к слабой зависимости КПД парогазовой установки от начального давления
- Температура уходящих газов остается выше минимально допустимой,

# Используем Map-Reduce

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter


# Map
map_template = """Ниже представлен список документов
{docs}
На основе этих документов определи, пожалуйста, основные темы 
Полезный ответ:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

reduce_template = """Ниже представлен набор кратких изложений:
{docs}
Возьми их и сведи в окончательное, объединенное краткое изложение основных тем. 
Полезный ответ:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=8000,
)

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

map_reduce_chain.run(split_docs)

'Основные темы, представленные в документах, включают влияние параметров пара на показатели комбинированной установки с простым паровым контуром, зависимость мощности паровой турбины от расхода пара, располагаемого перепада энтальпий на турбину и КПД паровой турбины, определение относительного расхода пара через уравнение теплового баланса испарительной и перегревательной поверхностей котла-утилизатора, влияние давления пара на относительный расход пара и температуру уходящих газов, а также влияние температуры пара перед паровой турбиной на влажность и КПД турбины.'

# 2 Поиск терминов

In [34]:
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.question_answering import load_qa_chain

note_path = "./notes/test_note.md"

loader = TextLoader(note_path, "utf-8")
docs = loader.load()

prompt_template = """Твоя задача найти в тексте термины без определений.
Игнорируй обычные слова. Ищи только термины, определения и сокращения, которым не хватает расшифровки.
Если таких нет, ничего не возвращай.
Текст: "{context}"
Ответ:"""
# prompt = PromptTemplate.from_template(prompt_template)
# stuff_chain = create_stuff_documents_chain(llm, prompt)
# stuff_chain.invoke({"context": docs})

qa_chain = load_qa_chain(llm, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)
abbreviations = qa_document_chain.run(
    input_document=docs[0].page_content,
    question="""Найди в тексте сокращения и аббревиатуры. Не выбирай нормальные слова. Выведи их в формате [термин_1, термин_2,
    термин_3, термин_4]. Не мотивируй свой ответ, не выводи другой текст.""",
)

abbreviations = list(set(abbreviations[abbreviations.find('['):abbreviations.find(']')].replace('"', '').replace("'", '').strip('][').split(', ')))

from langchain_community.utilities import GoogleSerperAPIWrapper
search = GoogleSerperAPIWrapper(serper_api_key=SERPER_API_KEY, gl='ru', hl='ru', k=5)

search_results = {}
for abb in abbreviations:
    search_results[abb] = search.run(f'{abb} это')

decodings = {}
for abbreviation, search_result in search_results.items():
    decoding = llm.invoke(f'''Найди расшифровку аббривиатуры "{abbreviation}" в тексте:
                          "{search_result}". Выведи только расшифровку. Не выводи опредеоление и другой текст. Не объясняй свой ответ.''').content
    decodings[abbreviation] = decoding

decodings

{'WAN': 'WAN - Глобальная компьютерная сеть (англ: Wide Area Network).',
 'LAN': 'LAN - Local Area Network',
 'MAC': 'Расшифровка аббревиатуры "MAC" в данном тексте - "Media Access Control" (надзор за доступом к среде).'}

# Roadmap

In [61]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Твоя задача - составить пользователю roadmap изучения указанной им темы/направления.
            Посоветуй, что и в каком порядке ему следует изучить, чтобы полностью разобраться в теме.
            Ответ пиши в формате списка. Составляй список в порядке возрастающей сложности. Следующие
            шаги должны опираться на знания из предыдущих. Для каждого пункта добавь 2-3 предложения, 
            где описываются, что входит в эту тему. Ограничь список 5-7 пунктами. Структура ответа: 
            1. Что изучить
                Состав программы
            2. Что изучить
                Состав программы
            
            Пример 1:
            Тема - Data Science
            Ответ:
            1. Теория вероятностей и описательная статистика
                категориальные и числовые данные, среднее значение, мода и медиана
                стандартное отклонение и дисперсия, ковариация, корреляция, асимметрия
                комбинаторика, события и их вероятности, классическая вероятность, условная вероятность
                формулы Байеса, Пуассона и Бернулли, локальная и интегральная теоремы Лапласа
                дискретные случайные величины, дискретные распределения (геометрическое, биномиальное, Пуассона)
                непрерывные случайные величины, непрерывные распределения (равномерное, показательное, нормальное)
                
            2. Языки программирования Python​
                типы данных: числа, строки, списки, множества, кортежи, циклы while и for, условия,
                их комбинации, функции, область видимости, lambda, рекурсия, декораторы, генераторы,
                вычислительная сложность, операции над структурами данных, стандартная библиотека,
                работа с ошибками и исключениями, try-except, raise, assert, работа с файлами: чтение,
                запись, сериализация, концепции ООП: полиморфизм, наследование, инкапсуляция
                
            3. Разные библиотеки, инструменты и техники Python для Data Science​
                pandas, numpy, scipy, matplotlib, scikit-learn, tensorflow
                
            ​4. SQL и базы данных​
                базовые концепции: таблицы, столбцы, строки и типы данных
                создание простых SELECT-запросов
                фильтрация с помощью WHERE и LIKE
                агрегирующие функции: COUNT, SUM, AVG, MAX/MIN
                группировка с помощью GROUP BY, HAVING
                объединение таблиц через JOIN
                CREATE TABLE для создания новых таблиц
                ALTER TABLE, DROP TABLE для изменения и удаления
                INSERT, UPDATE для добавления, изменения строк в таблице
                DELETE для удаления строк
                концепции базы данных (первичные и внешние ключи)
                создание новой БД при помощи CREATE DATABASE

            5. Машинное обучение​
                классическое обучение (регрессия, классификация, кластеризация,
                поиск правил, уменьшение размерности), ансамблевые методы ( стекинг,
                беггинг, бустинг), обучение с подрекплением, нейросети и глубокое обучение

            Пример 2:
            Тема - алгоритмы
            Ответ:
            1. Списки и хеширование
            2. Два указателя / Стек
            3. Бинарный поиск / Скользящее окно / Связанные листы
            4. Деревья
            5. Нагруженные деревья / Бектрекинг
            6. Куча / Графы / Одноразмерное динамическое программирование
            7. Интервалы / Жадный алгоритм / Двухразмерное динамическое программирование\n
            манипуляции с битами / Продвинутые графы 
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

learning_topic = 'ml system design'

roadmap = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content=learning_topic
            ),
        ],
    }
).content

print(roadmap)

1. Введение в машинное обучение
    - Основные понятия и определения машинного обучения
    - Различия между контролируемым, неконтролируемым и полуконтролируемым обучением
    - Примеры использования машинного обучения в различных областях

2. Архитектуры машинного обучения
    - Сверточные нейронные сети (CNN)
        - Работа с изображениями и видео
        - Фильтры, свертки, субдискретизация, пулинг
    - Рекуррентные нейронные сети (RNN)
        - Обработка последовательностей данных
        - Использование в задачах обработки естественного языка и прогнозировании временных рядов
    - Трансформеры
        - Attention Mechanism
        - Преимущества перед RNN

3. Оценка моделей машинного обучения
    - Валидация и тестирование моделей
    - Выбор метрик для оценки производительности модели
    - Кросс-валидация и время отклика

4. Инференс и оптимизация моделей
    - Введение в инференс
    - Фреймворки для инференса (TensorFlow, PyTorch, MXNet и др.)
    - Оптимизация моделей д